In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dm_control import suite
from dm_control import mujoco
from IPython.display import clear_output

    

In [2]:
env = suite.load(domain_name="fish", task_name="swim")
action_spec = env.action_spec()
observation_spec = env.observation_spec()
observation_shape = np.array([0])
time_step = env.reset()
print(type(observation_spec))
for (name, row) in observation_spec.items():
    print (name, observation_shape, row.shape)
    if(row.shape == ()):
        observation_shape[0] += 1
        continue
    print(row.shape)
    observation_shape[0] += row.shape[0]
observation_shape = (observation_shape[0],)
print(observation_shape)
print(action_spec.shape)

<class 'collections.OrderedDict'>
joint_angles [0] (7,)
(7,)
upright [7] ()
target [8] (3,)
(3,)
velocity [11] (13,)
(13,)
(24,)
(5,)


In [3]:
def observation2state(obs):
    result = np.array([])
    for(_, data) in obs.items():
        result = np.append(result, data)
    return result

In [13]:
from tensorforce.agents import PPOAgent
from tensorforce.agents import RandomAgent
agent = PPOAgent(
    states=dict(type='float', min_value=action_spec.minimum[0], max_value=action_spec.maximum[0], shape=observation_shape),
    actions=dict(type='float', min_value=action_spec.minimum[0], max_value=action_spec.maximum[0], shape=action_spec.shape),
    network=[
        dict(type='dense', size=128, activation='tanh'),
        dict(type='dense', size=64, activation='tanh'),
        dict(type='dense', size=16, activation='tanh')
    ],
    step_optimizer={
        "type": "adam",
        "learning_rate": 1e-4
    },
    entropy_regularization=0.01,
    batching_capacity=64,
    subsampling_fraction=0.1,
    optimization_steps=50,
    discount=0.99,
    likelihood_ratio_clipping=0.2,
    baseline_mode="states",
    baseline={
        "type":"mlp",
        "sizes": [32, 32]
    },
    baseline_optimizer={
        "type":"multi_step",
        "optimizer": {
            "type": "adam",
            "learning_rate": 1e-4
        },
        "num_steps": 5
    },
    update_mode={
        "unit": "episodes",
        "batch_size": 10,
        "frequency": 10
    },
    memory={
        "type": "latest",
        "include_next_states": False,
        "capacity": 2000
    }
)

In [14]:
j = 0

img = None
NUM_EPISODES = 25000

for i in range(NUM_EPISODES):
    time_step = env.reset()
    j = 0
    tot = 0
    while not time_step.last():
        state = observation2state(time_step.observation)
        action = agent.act(state)
        time_step = env.step(action)
        tot += time_step.reward
        agent.observe(reward=time_step.reward, terminal=time_step.last())
        if(j % 50 == 0 and i % 25 == 1):
            
            clear_output()
            img = plt.imshow(np.array(env.physics.render(480, 640)).reshape(480, 640, 3))
            plt.pause(0.5)
            
        j += 1
    if (j > 0):
        #tot /= j
        clear_output()
        print("for episode", i, ":", tot)

for episode 10208 : 58.34220142639916


KeyboardInterrupt: 